In [18]:
using LinearAlgebra

"""
Compute the largest eigenvalue and eigenvector of a (possibly complex) matrix `A` using the QR iteration method.

This method is well-suited for Hermitian matrices, for which the QR iteration will converge to a diagonal matrix whose diagonal elements are the eigenvalues. For general (non-Hermitian) matrices, QR iteration produces a Schur form (upper-triangular), and additional care is needed to interpret the results. Here we proceed as if the matrix is diagonalizable by this process.

Arguments:
- `A`: A square complex or real matrix. Ideally Hermitian if you expect a diagonal convergence.
- `num_iterations`: Maximum number of iterations to run.
- `tolerance`: Tolerance for convergence check.
- `round_digits`: Number of digits to round eigenvalues/eigenvectors for presentation.

Returns:
- `eigenvalue`: Leading eigenvalue (based on magnitude).
- `eigenvector`: Eigenvector corresponding to the leading eigenvalue.
"""
function qr_method_eigen(A; num_iterations::Int=1000, tolerance::Float64=1e-8, round_digits::Int=4)
    # Make a copy of A to avoid modifying the original
    A_current = copy(A) 
    n = size(A_current, 1)
    
    # Initially, P = I
    P = Matrix(I, n, n)
    
    # QR iteration
    for k = 1:num_iterations
        # Compute the QR factorization of A_current
        Q, R = qr(A_current)
        
        # Update A_k = R * Q
        A_next = R * Q
        
        # Update P = P * Q
        P = P * Q
        
        # Check convergence by examining the off-diagonal norm
        off_diag = A_next .- Diagonal(diag(A_next))
        if norm(off_diag) < tolerance
            A_current = A_next
            break
        end
        
        A_current = A_next
    end
    
    # Extract eigenvalues from the diagonal
    eigenvalues = diag(A_current)
    
    # Identify the index of the "largest" eigenvalue by magnitude
    index = argmax(abs.(eigenvalues))
    eigenval = eigenvalues[index]
     
    # The columns of P are the eigenvectors corresponding to these eigenvalues
    eigenvectors = P
    eigenvector = eigenvectors[:, index]
     
    # Rounding for presentation
    eigenval = round(eigenval, digits=round_digits)
    eigenvector = round.(eigenvector, digits=round_digits)
     
    return eigenval, eigenvector
end

# Example usage with a Hermitian (complex) matrix
A = [2.0+0im 1.0+0im; 1.0+0im 2.0+0im]
eigenval, eigenvector = qr_method_eigen(A)
println("Leading Eigenvalue: ", eigenval)
println("Leading Eigenvector: ", eigenvector)

# Compare with built-in eigen results
println("EXPECTED RESULTS")
eigen_result = eigen(A)

# Find the index of the eigenvalue with the largest magnitude
largest_index = argmax(abs.(eigen_result.values))

# Retrieve and print the largest eigenvalue and corresponding eigenvector
largest_eigenvalue = eigen_result.values[largest_index]
v1 = eigen_result.vectors[:, largest_index]
v1 = v1 / norm(v1)
println("Eigenvalue (built-in): ", largest_eigenvalue)
println("Eigenvector (built-in): ", v1)


Leading Eigenvalue: 3.0 + 0.0im
Leading Eigenvector: ComplexF64[0.7071 + 0.0im, 0.7071 + 0.0im]
EXPECTED RESULTS
Eigenvalue (built-in): 3.0
Eigenvector (built-in): ComplexF64[0.7071067811865476 + 0.0im, 0.7071067811865476 + 0.0im]


In [ ]:


# TESTING 
# Define a complex matrix
A = [3 3-2im; 3+2im 2]
 

# Compute the largest eigenvalue and eigenvector
eigenvalue, eigenvector = qr_method_eigen(A)
eigenvector = eigenvector / norm(eigenvector)
println("Eigenvalue: ", eigenvalue)
println("Eigenvector: ", round.(eigenvector, digits=4))

 

println();

# Compute eigenvalues and eigenvectors
println("EXPECTED RESULTS")
eigen_result = eigen(A)

# Find the index of the eigenvalue with the largest magnitude
largest_index = argmax(abs.(eigen_result.values)) # Use broadcasting with abs

# Retrieve the largest eigenvalue
largest_eigenvalue = eigen_result.values[largest_index]

# Print the largest eigenvalue
println("Eigenvalue: ", largest_eigenvalue)

# Retrieve and print the corresponding eigenvector
v1 = eigen_result.vectors[:, largest_index]
v1 = v1 / norm(v1)
println("Eigenvector: ", v1)


 
factor = (dot(v1, eigenvector))  
approx_equal = norm(eigenvector - factor*v1)
println("Difference: ", approx_equal)




# println();
# println();

 

# # Compute the largest singular value and singular vectors using your function
# singular_value, u, v = power_method_singular(A)
# u = u / norm(u)
# v = v / norm(v)

# println("Singular Value (Custom): ", singular_value)
# println("Left Singular Vector (Custom, u): ", u)
# println("Right Singular Vector (Custom, v): ", v)

# println("\nEXPECTED RESULTS")
# # Perform SVD using Julia's built-in function
# svd_result = svd(A)

# # Extract the largest singular value and corresponding singular vectors
# singular_value_builtin = svd_result.S[1]
# u_builtin = svd_result.U[:, 1]
# v_builtin = svd_result.V[:, 1]

# # Normalize Julia's singular vectors
# u_builtin = u_builtin / norm(u_builtin)
# v_builtin = v_builtin / norm(v_builtin)

# println("Singular Value (Built-in): ", singular_value_builtin)
# println("Left Singular Vector (Built-in, u_builtin): ", u_builtin)
# println("Right Singular Vector (Built-in, v_builtin): ", v_builtin)

# # Compute phase differences
# phase_u = angle(dot(u_builtin, u))
# phase_v = angle(dot(v_builtin, v))

# # Adjust your singular vectors to align phases
# u_adjusted = u * exp(-im * phase_u)
# v_adjusted = v * exp(-im * phase_v)

# # Compare the adjusted singular vectors with Julia's
# u_difference = norm(u_adjusted - u_builtin)
# v_difference = norm(v_adjusted - v_builtin)

# println("\nAfter adjusting for phase difference:")
# println("Difference between left singular vectors: ", u_difference)
# println("Difference between right singular vectors: ", v_difference)
 

Eigenvalue: 6.1401 + 0.0im
Eigenvector: ComplexF64[-0.7541 - 0.0im, -0.5464 - 0.3643im]

EXPECTED RESULTS
Eigenvalue: 6.1400549446402595
Eigenvector: ComplexF64[0.6274565591931813 - 0.4183043727954542im, 0.6567493570805042 + 0.0im]
Difference after adjusting phase factor: 3.194383531565379e-5
